In [1]:
###########################
#Case Study: Telecom Churn
#Author: Ambrish Gupta
#Date:June 08,2018
##########################    
    
####### import packages 
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

os.chdir("C:/Users/ajohn021/Documents/GitHub/IIITB_MachineLearning/Assignment- Telecom Churn")




In [2]:
### read the csv file
telecom_churn = pd.read_csv('telecom_churn_data.csv')
telecom_churn.info()
telecom_churn.shape
telecom_churn.head()
telecom_churn.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
count,9.999900e+04,99999.0,98981.0,98981.0,98981.0,99999.000000,99999.000000,99999.000000,99999.000000,96062.000000,...,99999.000000,25153.000000,25571.000000,26339.000000,25922.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
mean,7.001207e+09,109.0,0.0,0.0,0.0,282.987358,278.536648,279.154731,261.645069,132.395875,...,0.084581,0.914404,0.908764,0.890808,0.860968,1219.854749,68.170248,66.839062,60.021204,3.299373
std,6.956694e+05,0.0,0.0,0.0,0.0,328.439770,338.156291,344.474791,341.998630,297.207406,...,0.650457,0.279772,0.287950,0.311885,0.345987,954.733842,267.580450,271.201856,253.938223,32.408353
min,7.000000e+09,109.0,0.0,0.0,0.0,-2258.709000,-2014.045000,-945.808000,-1899.505000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000606e+09,109.0,0.0,0.0,0.0,93.411500,86.980500,84.126000,62.685000,7.380000,...,0.000000,1.000000,1.000000,1.000000,1.000000,467.000000,0.000000,0.000000,0.000000,0.000000
50%,7.001205e+09,109.0,0.0,0.0,0.0,197.704000,191.640000,192.080000,176.849000,34.310000,...,0.000000,1.000000,1.000000,1.000000,1.000000,863.000000,0.000000,0.000000,0.000000,0.000000
75%,7.001812e+09,109.0,0.0,0.0,0.0,371.060000,365.344500,369.370500,353.466500,118.740000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1807.500000,0.000000,0.000000,0.000000,0.000000
max,7.002411e+09,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,38805.617000,7376.710000,...,49.000000,1.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.210000,2618.570000


In [3]:
#replacing blank values with 0 - sabyasachi
telecom_churn.total_rech_amt_6.fillna(value=0, inplace=True)
telecom_churn.total_rech_amt_7.fillna(value=0, inplace=True)
telecom_churn.av_rech_amt_data_6.fillna(value=0, inplace=True)
telecom_churn.av_rech_amt_data_7.fillna(value=0, inplace=True)

In [4]:
#### Data Processing & data Cleaning 

### Renaming the vbc variables ####
telecom_churn['vbc_3g_6']=telecom_churn['jun_vbc_3g']
telecom_churn['vbc_3g_7']=telecom_churn['jul_vbc_3g']
telecom_churn['vbc_3g_8']=telecom_churn['aug_vbc_3g']
telecom_churn['vbc_3g_9']=telecom_churn['sep_vbc_3g']

telecom_churn.drop(['jun_vbc_3g','jul_vbc_3g','aug_vbc_3g','sep_vbc_3g'],axis=1,inplace=True)



###Finding high value accounts
#telecom_churn.drop(['totalrech_avg_goodphase'], axis = 1, inplace = True)
##telecom_churn['totalrech_avg_goodphase']=(telecom_churn['total_rech_amt_6']+telecom_churn['total_rech_amt_7'])/2
# sabyasachi - changed below average -  added av_rech_amt_data_6 and av_rech_amt_data_7
telecom_churn['totalrech_avg_goodphase']=(telecom_churn['total_rech_amt_6']+telecom_churn['total_rech_amt_7'] + telecom_churn['av_rech_amt_data_6'] + telecom_churn['av_rech_amt_data_7'])/4
telecom_churn = telecom_churn[telecom_churn['totalrech_avg_goodphase'] >  telecom_churn.totalrech_avg_goodphase.quantile(0.7)]


# making other missing values to 0 as well - sabyasachi
telecom_churn.max_rech_data_6.fillna(value=0, inplace=True)
telecom_churn.max_rech_data_7.fillna(value=0, inplace=True)
telecom_churn.max_rech_data_8.fillna(value=0, inplace=True)
telecom_churn.max_rech_data_9.fillna(value=0, inplace=True)

# cleaning up categorical variables - sabyasachi
telecom_churn.night_pck_user_6.fillna(value=-1, inplace=True)
telecom_churn.night_pck_user_7.fillna(value=-1, inplace=True)
telecom_churn.night_pck_user_8.fillna(value=-1, inplace=True)
telecom_churn.night_pck_user_9.fillna(value=-1, inplace=True)
telecom_churn.fb_user_6.fillna(value=-1, inplace=True)
telecom_churn.fb_user_7.fillna(value=-1, inplace=True)
telecom_churn.fb_user_8.fillna(value=-1, inplace=True)
telecom_churn.fb_user_9.fillna(value=-1, inplace=True)

###Adding the churn status
telecom_churn['churn_status']= ((telecom_churn.total_ic_mou_9 <=0) & (telecom_churn.total_og_mou_9 <=0)
                                & (telecom_churn.vol_2g_mb_9 <=0) & (telecom_churn.vol_3g_mb_9 <=0)).astype('int')

###91% are non-converters
telecom_churn['churn_status'].value_counts()/telecom_churn['churn_status'].shape[0]

#####removing all month 9 values,date columns and circleId
telecom_churn_sub=telecom_churn[telecom_churn.columns.drop(list(telecom_churn.filter(regex='_9')))]
telecom_churn_sub=telecom_churn_sub[telecom_churn_sub.columns.drop(list(telecom_churn_sub.filter(regex='date')))]
telecom_churn_sub.drop(['circle_id'],axis=1,inplace=True)
telecom_churn_sub.shape
print(telecom_churn_sub.shape)


(29949, 163)


In [5]:

##Replacing NA's with zero values for all the columns
telecom_churn_NAsToZero=telecom_churn_sub.apply(lambda x: x.fillna(0,axis=0))
print(telecom_churn_NAsToZero.shape)


(29949, 163)


In [6]:

####### Removing columns which have only 0 values
telecom_churn_NAsToZero.drop(['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou', 
                        'std_og_t2c_mou_6','std_og_t2c_mou_7','std_og_t2c_mou_8',
                        'std_ic_t2o_mou_6','std_ic_t2o_mou_7','std_ic_t2o_mou_8','mobile_number','totalrech_avg_goodphase'],axis=1,inplace=True)


print(telecom_churn_NAsToZero.shape)


(29949, 152)


In [7]:
#### Derived Variable
##############
telecom_churn_NAsToZero['arpu_tmp']=(telecom_churn_NAsToZero['arpu_6']+telecom_churn_NAsToZero['arpu_7'])/2
telecom_churn_NAsToZero['onnet_mou_tmp']=(telecom_churn_NAsToZero['onnet_mou_6']+telecom_churn_NAsToZero['onnet_mou_7'])/2
telecom_churn_NAsToZero['offnet_mou_tmp']=(telecom_churn_NAsToZero['offnet_mou_6']+telecom_churn_NAsToZero['offnet_mou_7'])/2
telecom_churn_NAsToZero['loc_og_t2t_mou_tmp']=(telecom_churn_NAsToZero['loc_og_t2t_mou_6']+telecom_churn_NAsToZero['loc_og_t2t_mou_7'])/2
telecom_churn_NAsToZero['loc_og_t2m_mou_tmp']=(telecom_churn_NAsToZero['loc_og_t2m_mou_6']+telecom_churn_NAsToZero['loc_og_t2m_mou_7'])/2
telecom_churn_NAsToZero['loc_og_t2f_mou_tmp']=(telecom_churn_NAsToZero['loc_og_t2f_mou_6']+telecom_churn_NAsToZero['loc_og_t2f_mou_7'])/2
telecom_churn_NAsToZero['loc_og_t2c_mou_tmp']=(telecom_churn_NAsToZero['loc_og_t2c_mou_6']+telecom_churn['loc_og_t2c_mou_7'])/2
telecom_churn_NAsToZero['loc_og_mou_tmp']=(telecom_churn_NAsToZero['loc_og_mou_6']+telecom_churn_NAsToZero['loc_og_mou_7'])/2
telecom_churn_NAsToZero['std_og_t2t_mou_tmp']=(telecom_churn_NAsToZero['std_og_t2t_mou_6']+telecom_churn_NAsToZero['std_og_t2t_mou_7'])/2
telecom_churn_NAsToZero['std_og_t2m_mou_tmp']=(telecom_churn['std_og_t2m_mou_6']+telecom_churn_NAsToZero['std_og_t2m_mou_7'])/2
telecom_churn_NAsToZero['std_og_mou_tmp']=(telecom_churn_NAsToZero['std_og_mou_6']+telecom_churn_NAsToZero['std_og_mou_7'])/2
telecom_churn_NAsToZero['total_og_mou_tmp']=(telecom_churn_NAsToZero['total_og_mou_6']+telecom_churn_NAsToZero['total_og_mou_7'])/2
telecom_churn_NAsToZero['loc_ic_t2t_mou_tmp']=(telecom_churn_NAsToZero['loc_ic_t2t_mou_6']+telecom_churn_NAsToZero['loc_ic_t2t_mou_7'])/2
telecom_churn_NAsToZero['loc_ic_t2m_mou_tmp']=(telecom_churn_NAsToZero['loc_ic_t2m_mou_6']+telecom_churn_NAsToZero['loc_ic_t2m_mou_7'])/2
telecom_churn_NAsToZero['loc_ic_mou_tmp']=(telecom_churn_NAsToZero['loc_ic_mou_6']+telecom_churn_NAsToZero['loc_ic_mou_7'])/2
telecom_churn_NAsToZero['std_ic_t2t_mou_tmp']=(telecom_churn_NAsToZero['std_ic_t2t_mou_6']+telecom_churn_NAsToZero['std_ic_t2t_mou_7'])/2
telecom_churn_NAsToZero['std_ic_t2m_mou_tmp']=(telecom_churn_NAsToZero['std_ic_t2m_mou_6']+telecom_churn_NAsToZero['std_ic_t2m_mou_7'])/2
telecom_churn_NAsToZero['std_ic_mou_tmp']=(telecom_churn_NAsToZero['std_ic_mou_6']+telecom_churn_NAsToZero['std_ic_mou_7'])/2
telecom_churn_NAsToZero['total_ic_mou_tmp']=(telecom_churn_NAsToZero['total_ic_mou_6']+telecom_churn_NAsToZero['total_ic_mou_7'])/2
telecom_churn_NAsToZero['isd_ic_mou_tmp']=(telecom_churn_NAsToZero['isd_ic_mou_6']+telecom_churn_NAsToZero['isd_ic_mou_7'])/2
telecom_churn_NAsToZero['total_rech_num_tmp']=(telecom_churn_NAsToZero['total_rech_num_6']+telecom_churn_NAsToZero['total_rech_num_7'])/2
telecom_churn_NAsToZero['total_rech_amt_tmp']=(telecom_churn_NAsToZero['total_rech_amt_6']+telecom_churn_NAsToZero['total_rech_amt_7'])/2
telecom_churn_NAsToZero['max_rech_amt_tmp']=(telecom_churn_NAsToZero['max_rech_amt_6']+telecom_churn_NAsToZero['max_rech_amt_7'])/2
telecom_churn_NAsToZero['last_day_rch_amt_tmp']=(telecom_churn_NAsToZero['last_day_rch_amt_6']+telecom_churn_NAsToZero['last_day_rch_amt_7'])/2
telecom_churn_NAsToZero['max_rech_data_tmp']=(telecom_churn_NAsToZero['max_rech_data_6']+telecom_churn_NAsToZero['max_rech_data_7'])/2
telecom_churn_NAsToZero['av_rech_amt_data_tmp']=(telecom_churn_NAsToZero['av_rech_amt_data_6']+telecom_churn_NAsToZero['av_rech_amt_data_7'])/2
telecom_churn_NAsToZero['vol_2g_mb_tmp']=(telecom_churn_NAsToZero['vol_2g_mb_6']+telecom_churn_NAsToZero['vol_2g_mb_7'])/2
telecom_churn_NAsToZero['vol_3g_mb_tmp']=(telecom_churn_NAsToZero['vol_3g_mb_6']+telecom_churn_NAsToZero['vol_3g_mb_7'])/2
telecom_churn_NAsToZero['arpu_3g_tmp']=(telecom_churn_NAsToZero['arpu_3g_6']+telecom_churn_NAsToZero['arpu_3g_7'])/2
telecom_churn_NAsToZero['arpu_2g_tmp']=(telecom_churn_NAsToZero['arpu_2g_6']+telecom_churn_NAsToZero['arpu_2g_7'])/2
telecom_churn_NAsToZero['vbc_3g_tmp']=(telecom_churn_NAsToZero['vbc_3g_6']+telecom_churn_NAsToZero['vbc_3g_7'])/2



In [8]:

### Total Roaming  IC & OG  ### 
telecom_churn_NAsToZero['total_roam_ic_mou']  =telecom_churn_NAsToZero['roam_ic_mou_6'] + telecom_churn_NAsToZero['roam_ic_mou_7']+telecom_churn_NAsToZero['roam_ic_mou_8']
telecom_churn_NAsToZero['total_roam_og_mou']=telecom_churn_NAsToZero['roam_og_mou_6'] + telecom_churn_NAsToZero['roam_og_mou_7']+telecom_churn_NAsToZero['roam_og_mou_8']

### Total IC & OG  ### 
telecom_churn_NAsToZero['total_ic_mou']=telecom_churn_NAsToZero['total_ic_mou_6'] + telecom_churn_NAsToZero['total_ic_mou_7']+telecom_churn_NAsToZero['total_ic_mou_8']
telecom_churn_NAsToZero['total_og_mou']=telecom_churn_NAsToZero['total_og_mou_6'] + telecom_churn_NAsToZero['total_og_mou_7']+telecom_churn_NAsToZero['total_og_mou_8']

#### Roaming Percent

#telecom_churn_NAsToZero['total_roam_ic_pct']=telecom_churn_NAsToZero['total_roam_ic_mou']/telecom_churn_NAsToZero['total_ic_mou']
#telecom_churn_NAsToZero['total_roam_og_pct']=telecom_churn_NAsToZero['total_roam_og_mou']/telecom_churn_NAsToZero['total_og_mou']

Var_list=telecom_churn_NAsToZero.columns
print(Var_list.shape)

(187,)


In [9]:
telecom_churn_NAsToZero['arpu_flag']=(telecom_churn_NAsToZero['arpu_8']/telecom_churn_NAsToZero['arpu_tmp']).round(2)
telecom_churn_NAsToZero['onnet_mou_flag']=(telecom_churn_NAsToZero['onnet_mou_8']/telecom_churn_NAsToZero['onnet_mou_tmp']).round(2)
telecom_churn_NAsToZero['offnet_mou_flag']=(telecom_churn_NAsToZero['offnet_mou_8']/telecom_churn_NAsToZero['offnet_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_og_t2t_mou_flag']=(telecom_churn_NAsToZero['loc_og_t2t_mou_8']/telecom_churn_NAsToZero['loc_og_t2t_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_og_t2m_mou_flag']=(telecom_churn_NAsToZero['loc_og_t2m_mou_8']/telecom_churn_NAsToZero['loc_og_t2m_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_og_t2f_mou_flag']=(telecom_churn_NAsToZero['loc_og_t2f_mou_8']/telecom_churn_NAsToZero['loc_og_t2f_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_og_t2c_mou_flag']=(telecom_churn_NAsToZero['loc_og_t2c_mou_8']/telecom_churn_NAsToZero['loc_og_t2c_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_og_mou_flag']=(telecom_churn_NAsToZero['loc_og_mou_8']/telecom_churn_NAsToZero['loc_og_mou_tmp']).round(2)
telecom_churn_NAsToZero['std_og_t2t_mou_flag']=(telecom_churn_NAsToZero['std_og_t2t_mou_8']/telecom_churn_NAsToZero['std_og_t2t_mou_tmp']).round(2)
telecom_churn_NAsToZero['std_og_t2m_mou_flag']=(telecom_churn['std_og_t2m_mou_8']/telecom_churn_NAsToZero['std_og_t2m_mou_tmp']).round(2)
telecom_churn_NAsToZero['std_og_mou_flag']=(telecom_churn_NAsToZero['std_og_mou_8']/telecom_churn_NAsToZero['std_og_mou_tmp']).round(2)
telecom_churn_NAsToZero['total_og_mou_flag']=(telecom_churn_NAsToZero['total_og_mou_8']/telecom_churn_NAsToZero['total_og_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_ic_t2t_mou_flag']=(telecom_churn_NAsToZero['loc_ic_t2t_mou_8']/telecom_churn_NAsToZero['loc_ic_t2t_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_ic_t2m_mou_flag']=(telecom_churn_NAsToZero['loc_ic_t2m_mou_8']/telecom_churn_NAsToZero['loc_ic_t2m_mou_tmp']).round(2)
telecom_churn_NAsToZero['loc_ic_mou_flag']=(telecom_churn_NAsToZero['loc_ic_mou_8']/telecom_churn_NAsToZero['loc_ic_mou_tmp']).round(2)
telecom_churn_NAsToZero['std_ic_t2t_mou_flag']=(telecom_churn_NAsToZero['std_ic_t2t_mou_8']/telecom_churn_NAsToZero['std_ic_t2t_mou_tmp']).round(2)
telecom_churn_NAsToZero['std_ic_t2m_mou_flag']=(telecom_churn_NAsToZero['std_ic_t2m_mou_8']/telecom_churn_NAsToZero['std_ic_t2m_mou_tmp']).round(2)
telecom_churn_NAsToZero['std_ic_mou_flag']=(telecom_churn_NAsToZero['std_ic_mou_8']/telecom_churn_NAsToZero['std_ic_mou_tmp']).round(2)
telecom_churn_NAsToZero['total_ic_mou_flag']=(telecom_churn_NAsToZero['total_ic_mou_8']/telecom_churn_NAsToZero['total_ic_mou_tmp']).round(2)
telecom_churn_NAsToZero['isd_ic_mou_flag']=(telecom_churn_NAsToZero['isd_ic_mou_8']/telecom_churn_NAsToZero['isd_ic_mou_tmp']).round(2)
telecom_churn_NAsToZero['total_rech_num_flag']=(telecom_churn_NAsToZero['total_rech_num_8']/telecom_churn_NAsToZero['total_rech_num_tmp']).round(2)
telecom_churn_NAsToZero['total_rech_amt_flag']=(telecom_churn_NAsToZero['total_rech_amt_8']/telecom_churn_NAsToZero['total_rech_amt_tmp']).round(2)
telecom_churn_NAsToZero['max_rech_amt_flag']=(telecom_churn_NAsToZero['max_rech_amt_8']/telecom_churn_NAsToZero['max_rech_amt_tmp']).round(2)
telecom_churn_NAsToZero['last_day_rch_amt_flag']=(telecom_churn_NAsToZero['last_day_rch_amt_8']/telecom_churn_NAsToZero['last_day_rch_amt_tmp']).round(2)
telecom_churn_NAsToZero['max_rech_data_flag']=(telecom_churn_NAsToZero['max_rech_data_8']/telecom_churn_NAsToZero['max_rech_data_tmp']).round(2)
telecom_churn_NAsToZero['av_rech_amt_data_flag']=(telecom_churn_NAsToZero['av_rech_amt_data_8']/telecom_churn_NAsToZero['av_rech_amt_data_tmp']).round(2)
telecom_churn_NAsToZero['vol_2g_mb_flag']=(telecom_churn_NAsToZero['vol_2g_mb_8']/telecom_churn_NAsToZero['vol_2g_mb_tmp']).round(2)
telecom_churn_NAsToZero['vol_3g_mb_flag']=(telecom_churn_NAsToZero['vol_3g_mb_8']/telecom_churn_NAsToZero['vol_3g_mb_tmp']).round(2)
telecom_churn_NAsToZero['arpu_3g_flag']=(telecom_churn_NAsToZero['arpu_3g_8']/telecom_churn_NAsToZero['arpu_3g_tmp']).round(2)
telecom_churn_NAsToZero['arpu_2g_flag']=(telecom_churn_NAsToZero['arpu_2g_8']/telecom_churn_NAsToZero['arpu_2g_tmp']).round(2)
telecom_churn_NAsToZero['vbc_3g_flag']=(telecom_churn_NAsToZero['vbc_3g_8']/telecom_churn_NAsToZero['vbc_3g_tmp']).round(2)

In [15]:
##### Data with key variables +  derived variables only
print(telecom_churn_NAsToZero.shape)
New_Var_list=telecom_churn_NAsToZero.columns
print(New_Var_list.shape)
Add_Var=["aon","churn_status"]

Imp_Var_list=list(set(New_Var_list)) 
#Imp_Var_list=list(set(New_Var_list)-set(Var_list)) 

Final_Var_list=Imp_Var_list
#Final_Var_list=Imp_Var_list+Add_Var
print(Final_Var_list)

Modeling_Data=telecom_churn_NAsToZero[Final_Var_list]

Modeling_Data=Modeling_Data.replace([np.inf, -np.inf], 2)

Modeling_Data=Modeling_Data.apply(lambda x: x.fillna(0,axis=0))
#Modeling_Data.to_csv("Modeling_Data.csv")

##### Checking the Mean
#Modeling_Data_mean=Modeling_Data.groupby('churn_status')[Modeling_Data.columns].mean()
#Modeling_Data_mean.to_csv("Modeling_Data_mean.csv")



(29949, 218)
(218,)
['vol_3g_mb_tmp', 'total_rech_data_8', 'monthly_2g_7', 'last_day_rch_amt_tmp', 'std_og_t2t_mou_8', 'sachet_3g_6', 'std_og_t2t_mou_tmp', 'av_rech_amt_data_flag', 'loc_og_t2f_mou_tmp', 'std_ic_t2t_mou_flag', 'max_rech_data_7', 'loc_ic_t2f_mou_7', 'loc_og_t2t_mou_6', 'isd_ic_mou_8', 'arpu_2g_7', 'count_rech_2g_8', 'arpu_2g_6', 'std_og_t2f_mou_6', 'vol_2g_mb_6', 'loc_ic_mou_6', 'max_rech_data_tmp', 'sachet_3g_8', 'count_rech_3g_7', 'total_ic_mou_flag', 'spl_og_mou_7', 'loc_ic_t2t_mou_7', 'total_og_mou_7', 'std_ic_t2f_mou_8', 'count_rech_2g_6', 'onnet_mou_tmp', 'total_rech_amt_tmp', 'onnet_mou_8', 'total_og_mou_6', 'isd_ic_mou_tmp', 'std_og_mou_flag', 'total_rech_num_8', 'std_og_t2f_mou_7', 'max_rech_data_flag', 'arpu_8', 'max_rech_data_6', 'onnet_mou_6', 'loc_ic_t2m_mou_8', 'aon', 'arpu_2g_tmp', 'vol_2g_mb_8', 'arpu_3g_7', 'isd_ic_mou_6', 'sachet_2g_6', 'loc_og_t2m_mou_8', 'ic_others_6', 'loc_ic_t2m_mou_6', 'total_ic_mou_6', 'og_others_8', 'total_rech_data_6', 'isd_og_m

In [16]:

# splitting into X and y
X = Modeling_Data.drop("churn_status", axis = 1)
Y = Modeling_Data.churn_status.astype(int)

###Min-Max Scaling applied
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
X_scaled= pd.DataFrame(X_scaled)
X_scaled=scale(X)
####Splitting 70-30
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size = 0.3, random_state = 100)



print(X_train.shape)
print(X_test.shape)

####Recall seems to be the right metric since we are interested in identifying more churners correctly
##Recall <- Number of churners identified/Total number of churners
#####Random Forest/SVM/XG-Boost algorithms should give the best results for prediction.


(20964, 217)
(8985, 217)


In [17]:
##PCA capturing ~  99% of variance(50 components) ##make sure flag is set in the beginning
pca = PCA(svd_solver='randomized', random_state=100)
pca.fit(X_train)
pca.explained_variance_ratio_
pd.DataFrame(pca.explained_variance_ratio_).to_csv('variance_explained.csv')
#fig = plt.figure(figsize = (40,8))
#plt.plot(np.cumsum(pca.explained_variance_ratio_))
#plt.xlabel('number of components')
#plt.ylabel('cumulative explained variance')
#plt.show()


In [18]:
#pca_final = IncrementalPCA(n_components=50)

#X_train=pca_final.fit_transform(X_train)
#X_test=pca_final.fit_transform(X_test)

#X_train.shape
#
#X_test.shape

In [19]:
#################################################################################

#####################################ADA-BOOST##########################################

#################################################################################

# base estimator
tree = DecisionTreeClassifier(max_depth=3,class_weight='balanced')

# adaboost with the tree as base estimator
adaboost_model_1 = AdaBoostClassifier(
    base_estimator=tree,
    n_estimators=200,
    learning_rate=1,
    algorithm="SAMME")

adaboost_model_1.fit(X_train, y_train)
y_pred = adaboost_model_1.predict(X_test)
# metrics
print(metrics.confusion_matrix(y_test, y_pred), "\n")
print("accuracy", metrics.accuracy_score(y_test, y_pred))
print("precision", metrics.precision_score(y_test, y_pred))
print("sensitivity/recall", metrics.recall_score(y_test, y_pred))
print("ROC",metrics.roc_auc_score(y_test, y_pred))
print("F1Score",metrics.f1_score(y_test, y_pred))


[[7629  623]
 [ 199  534]] 

accuracy 0.9085141903171954
precision 0.46153846153846156
sensitivity/recall 0.7285129604365621
ROC 0.8265080555939477
F1Score 0.5650793650793651


In [21]:
# parameter grid
param_grid = {"base_estimator__max_depth" : [3, 10],
              "n_estimators": [200, 400, 600]
             }

# base estimator
tree = DecisionTreeClassifier(class_weight='balanced')

# adaboost with the tree as base estimator
adaboost_model_2 = AdaBoostClassifier(
    base_estimator=tree,
    learning_rate=1,
    algorithm="SAMME")

folds = 3
grid_search_adaboost_model_2 = GridSearchCV(adaboost_model_2, 
                               cv = folds,
                               param_grid=param_grid, 
                               scoring = 'f1', 
                               return_train_score=True,                         
                               verbose = 1)


grid_search_adaboost_model_2.fit(X_train, y_train)



Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 105.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'base_estimator__max_depth': [3, 10], 'n_estimators': [200, 400, 600]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=1)

In [22]:
y_pred = grid_search_adaboost_model_2.predict(X_test)
# metrics
print(metrics.confusion_matrix(y_test, y_pred), "\n")
print("accuracy", metrics.accuracy_score(y_test, y_pred))
print("precision", metrics.precision_score(y_test, y_pred))
print("sensitivity/recall", metrics.recall_score(y_test, y_pred))
print("ROC",metrics.roc_auc_score(y_test, y_pred))

[[8158   94]
 [ 421  312]] 

accuracy 0.9426822481914302
precision 0.7684729064039408
sensitivity/recall 0.4256480218281037
ROC 0.707128421965918
